In [9]:
import telebot
from telebot.types import Message

#get API token and start bot
f=open("API_TOKEN","r")
lines=f.readlines()
API_TOKEN = lines[0]
f.close()

bot = telebot.TeleBot(API_TOKEN)

# Keep track of user data: user_id -> {'prompt_msg_id': int, 'session_text': str, 'accepted_msg_id': int}
user_sessions = {}

@bot.message_handler(commands=['start'])
def handle_start(message: Message):
    prompt = bot.send_message(message.chat.id, "Write the details of a session you want to facilitate as a reply")
    user_sessions[message.from_user.id] = {'prompt_msg_id': prompt.message_id}

@bot.message_handler(func=lambda m: m.reply_to_message is not None and not m.text.startswith("/"))
def handle_session_reply(message: Message):
    user_id = message.from_user.id
    replied_to = message.reply_to_message

    session_info = user_sessions.get(user_id)
    if session_info and replied_to.message_id == session_info.get('prompt_msg_id'):
        # Save the user's session message
        session_info['session_text'] = message.text
        accepted_msg = bot.reply_to(message, "Step 1. Start meeting \n Step 2. End meeting \n Step 3. ????? \n Step 4. Profit! \n Reply to this message with /transcript <transcript> so we can analyze your meeting")
        session_info['accepted_msg_id'] = accepted_msg.message_id

@bot.message_handler(commands=['transcript'])
def handle_transcript(message: Message):
    user_id = message.from_user.id
    replied_to = message.reply_to_message

    session_info = user_sessions.get(user_id)
    if session_info and replied_to and replied_to.message_id == session_info.get('accepted_msg_id'):
        # Get the <text> from the /transcript command
        parts = message.text.split(' ', 1)
        if len(parts) < 2:
            bot.reply_to(message, "Please provide text after /transcript.")
            return

        transcript_text = parts[1]
        original = session_info.get('session_text', '[No session text found]')
        reply = f"📝 *Session Details:*\n{original}\n\n🗒️ *Transcript:*\n{transcript_text}"
        bot.reply_to(message, reply, parse_mode='Markdown')

bot.infinity_polling()


2025-04-16 22:40:01,295 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2025-04-16 22:40:01,296 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
